# ImageNet Performance
> Performance data for the ImageNet benchmark

- toc: true 
- badges: true
- comments: true
- categories: [l3, performance, ImageNet]
- image: images/imagenet_logo.png

In [ ]:
csv_path = "./data/cleaned_csv/backup.csv"

## Roofline

## L3 Performance

In [1]:
#hide
import pandas as pd
import numpy as np
import altair as alt

W = 600
H = 480

In [ ]:
#hide
#%writefile scripts/utils.py
def norm_by_group(df, column, group_col):
    """ Normalizes pandas series by group """
    df["norm-"+column] = df.groupby(group_col)[column].apply(lambda x: (x / x.max()))
    return df

def select_color(sel, column):
    """ Easy way to set colors based on selection for altair plots
    """
    return alt.condition(sel, 
                      alt.Color(column),
                      alt.value('lightgray'))

def get_pareto_df(df, groupcol, xcol, ycol):
    pareto_line_df = df.groupby(groupcol)[xcol].max().to_frame("x")
    pareto_line_df['y'] = df.groupby(groupcol)[ycol].agg(lambda x: x.value_counts().index[0])
    pareto_line_df.sort_values('y', ascending=False, inplace=True)
    pareto_line_df['x'] = pareto_line_df.x.cummax()
    pareto_line_df.drop_duplicates('x', keep='first', inplace=True)
    pareto_line_df['group'] = pareto_line_df.index
    return pareto_line_df

def label_point(x, y, val, ax, rot=0):
    """ from https://stackoverflow.com/questions/46027653/adding-labels-in-x-y-scatter-plot-with-seaborn"""
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.02, point['y'], str(point['val']), rotation=rot)

In [ ]:
#hide
master_df = pd.read_csv(csv_path)
is_maxp = lambda row: row.HWType != "TX2" or row["Op mode"].split(",")[0] == "maxp"
maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
imagenet_df = maxp_df[maxp_df.NN_Topology.isin(['GNv1','RN50','MNv1']) & maxp_df["top1 [%]"].notna() & maxp_df['lat-comp'].notna()]
bad_precisions = ["FP"+str(i) for i in range(17,24)]
imagenet_df.Precision = imagenet_df.Precision.apply(lambda x: 'FP16' if x in bad_precisions else x)
imagenet_df["hw_quant_prun_net"] = imagenet_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.PruningFactor, r.NN_Topology]), axis=1)
imagenet_df["PruningFactor"] = imagenet_df["PruningFactor"].str.strip("%").astype(float)
norm_by_group(imagenet_df, "lat-comp", "NN_Topology");
imagenet_df["quant_model"] = imagenet_df.Precision + '_' + imagenet_df.HWType
imagenet_df.rename(columns={"top1 [%]": "top1"}, inplace=True)
imagenet_df["hw_precision_net_prun"] = imagenet_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.NN_Topology, str(r.PruningFactor)]), axis=1)

## Table Statistics

In [ ]:
#hide_input
imagenet_df.describe()

## Line Plot

In [ ]:
#hide_input

sel = alt.selection_multi(fields=["hw_quant_prun_net"], bind="legend")
fig25_dot = alt.Chart(imagenet_df).mark_point().encode(
    x='lat-comp',
    y=alt.Y('fps-comp', scale=alt.Scale(type="log")),
    color=select_color(sel, 'hw_quant_prun_net:N'),
    tooltip=['Op mode', 'fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
)
fig25_line = alt.Chart(imagenet_df).mark_line().encode(
    x='lat-comp',
    y='fps-comp',
    color=select_color(sel, 'hw_quant_prun_net:N'),
    tooltip=['Op mode', 'fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
)

fig = (fig25_dot+fig25_line).properties(
    title="Latency versus Performance for Pruned and Quantized Imagenet Classifier Variants",
    width=W,
    height=H,
).add_selection(sel).interactive()

fig

## Boxplots

In [ ]:
#hide_input
box1 = alt.Chart(imagenet_df).mark_boxplot().encode(
    x='PruningFactor:O',
    y=alt.Y("lat-comp", scale=alt.Scale(type="log")),
    color='PruningFactor:O',
).facet(column="quant_model").properties(
    title="Latency by Hardware/Framework and Pruning for imagenet"
).interactive()
box1

In [ ]:
#hide_input
box1 = alt.Chart(imagenet_df).mark_boxplot().encode(
    x='PruningFactor:O',
    y=alt.Y("fps-comp", scale=alt.Scale(type="log")),
    color='PruningFactor:O',
).facet(column="quant_model").properties(
    title="Throughput by Hardware/Framework and Pruning for imagenet",
).interactive()
box1

## Pareto

In [ ]:
#hide_input
imagenet_pareto = get_pareto_df(imagenet_df, 'hw_precision_net_prun', 'fps-comp', 'top1')
imagenet_lines = alt.Chart(imagenet_df).mark_line(point=True).encode(
    x="fps-comp",
    y=alt.Y("top1:Q", scale=alt.Scale(zero=False)),
    color=alt.Color("hw_precision_net_prun", legend=alt.Legend(columns=1)),
    tooltip=["HWType", "Precision", "PruningFactor", "batch/thread/stream", "top1", "fps-comp"],
)
imagenet_pareto_plot = alt.Chart(imagenet_pareto).mark_line().encode(
    x="x",
    y=alt.Y("y", scale=alt.Scale(zero=False)),
)
(imagenet_lines+imagenet_pareto_plot).interactive().properties(
    width=W,
    height=H,
    title="ImageNet Cassification Design Space: Accuracy versus Performance"
)

# Performance Predictions for Imagenet

In [2]:
#hide_input
%run scripts/heatmaps.py  #run the heatmaps script
#load cifar10 dataset and plot it
heatmap(pd.read_csv("data/processed_csv/performance_prediction_imagenet.csv"), 'lightgrey','Performance predictions for Imagenet')

alt.LayerChart(...)